In [30]:
text = "This is a sentence for trying unigram,bigrams creation."

In [31]:
from nltk import word_tokenize, ngrams

In [32]:
list(ngrams(word_tokenize(text), 2))

[('This', 'is'),
 ('is', 'a'),
 ('a', 'sentence'),
 ('sentence', 'for'),
 ('for', 'trying'),
 ('trying', 'unigram'),
 ('unigram', ','),
 (',', 'bigrams'),
 ('bigrams', 'creation'),
 ('creation', '.')]

In [33]:
list(ngrams(text, 2))

[('T', 'h'),
 ('h', 'i'),
 ('i', 's'),
 ('s', ' '),
 (' ', 'i'),
 ('i', 's'),
 ('s', ' '),
 (' ', 'a'),
 ('a', ' '),
 (' ', 's'),
 ('s', 'e'),
 ('e', 'n'),
 ('n', 't'),
 ('t', 'e'),
 ('e', 'n'),
 ('n', 'c'),
 ('c', 'e'),
 ('e', ' '),
 (' ', 'f'),
 ('f', 'o'),
 ('o', 'r'),
 ('r', ' '),
 (' ', 't'),
 ('t', 'r'),
 ('r', 'y'),
 ('y', 'i'),
 ('i', 'n'),
 ('n', 'g'),
 ('g', ' '),
 (' ', 'u'),
 ('u', 'n'),
 ('n', 'i'),
 ('i', 'g'),
 ('g', 'r'),
 ('r', 'a'),
 ('a', 'm'),
 ('m', ','),
 (',', 'b'),
 ('b', 'i'),
 ('i', 'g'),
 ('g', 'r'),
 ('r', 'a'),
 ('a', 'm'),
 ('m', 's'),
 ('s', ' '),
 (' ', 'c'),
 ('c', 'r'),
 ('r', 'e'),
 ('e', 'a'),
 ('a', 't'),
 ('t', 'i'),
 ('i', 'o'),
 ('o', 'n'),
 ('n', '.')]

In [16]:
import pandas as pd

names_df = pd.read_csv("nepali_names.csv",sep = "\t")

names_df=names_df.sample(frac=1).reset_index(drop=True)



rough_words_df = pd.read_csv("rough_words.csv",sep = "\t")

rough_words_df=rough_words_df.sample(frac=1).reset_index(drop=True)


possible_text_df = pd.read_csv("possible_text_entry.csv",sep = "\t")

possible_text_df=possible_text_df.sample(frac=1).reset_index(drop=True)



english_typed_nepali_df = pd.read_csv("english_typed_nepali_word.csv",sep="\t")

english_typed_nepali_df=english_typed_nepali_df.sample(frac=1).reset_index(drop=True)



nepali_chat_df = pd.read_csv("nepali_chat_text.csv",sep="\t")

nepali_chat_df=nepali_chat_df.sample(frac=1).reset_index(drop=True)


nepali_chat_df = nepali_chat_df[33000:]


non_name_df_list = [rough_words_df,possible_text_df,nepali_chat_df,english_typed_nepali_df]
non_name_df = pd.concat(non_name_df_list)



final_df = names_df.append(non_name_df, ignore_index=True)

final_df=final_df.sample(frac=1).reset_index(drop=True)

In [17]:
final_df.shape

(23782, 2)

In [18]:
final_df.dtypes

TEXT     object
LABEL    object
dtype: object

In [19]:
final_df.isna().sum()

TEXT     0
LABEL    0
dtype: int64

In [20]:
final_df.head()

,TEXT,LABEL
0,Sonim Kumar,NAME
1,Dayaram,NAME
2,Dendi Aditya,NAME
3,Upadhya Jung,NAME
4,Abhash Aditya,NAME


In [21]:
type(final_df["TEXT"][255])

str

final_df["TEXT"] = final_df["TEXT"].to_string()

In [22]:
final_df.loc[:,"TEXT"] = final_df.TEXT.apply(lambda x : str(x))

In [23]:
final_df.loc[:,"TEXT"] = final_df.TEXT.apply(lambda x : str.lower(x))

In [24]:
for i in final_df["TEXT"]:
    if type(i)!=str:
        print("bad")

In [25]:
final_df.head(n=15)

,TEXT,LABEL
0,sonim kumar,NAME
1,dayaram,NAME
2,dendi aditya,NAME
3,upadhya jung,NAME
4,abhash aditya,NAME
5,sadashyaharuprati,NOTNAME
6,surakshit man,NAME
7,sudep bachu ayohrishab,NOTNAME
8,bishowram,NAME
9,rohin,NAME


In [26]:
final_df["TEXT"] = final_df["TEXT"][0:2000]

In [27]:
final_df["TEXT"][15:25]

15                        sumin
16               hansha bahadur
17                      trideep
18                       tansen
19            oye pulchowk xams
20    padhya xaina dabhijeet ke
21                   kabita man
22                  min chandra
23                     adhikrit
24                       uttano
Name: TEXT, dtype: object

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df["TEXT"], final_df["LABEL"], 
                                                    test_size=.33, random_state=17)

In [29]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vocabulary_size = 2000
tokenizer = Tokenizer(char_level=False, oov_token='UNK',num_words = vocabulary_size)
tokenizer.fit_on_texts(final_df["TEXT"])
sequences = tokenizer.texts_to_sequences(final_df['TEXT'])
data = pad_sequences(sequences,maxlen = 20)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
tokenizer.index_word

In [ ]:
final_df["LABEL"] = final_df["LABEL"].replace(["NAME" , "NOTNAME"," NOTNAME"] , [1 , 0 , 0])

In [ ]:
labels = final_df["LABEL"]

In [ ]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense,  LSTM, Conv1D, MaxPooling1D, Dropout, Activation
import numpy as np
from keras.layers import BatchNormalization

In [ ]:
model2 = Sequential()
model2.add(Embedding(41, 50, input_length=20))
model2.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))
model2.add(Dense(50, activation="tanh"))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))
model2.add(Dense(50, activation="tanh"))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))
model2.add(Dense(1, activation="sigmoid"))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(data, np.array(labels), validation_split=0.4, epochs=20)

In [ ]:
### Sample prediction
text1 = "mula"
text= np.array([text1])
text= tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen=50)
prediction = model2.predict(text)
print(prediction)

In [ ]:
test_df = pd.read_csv("test_data.csv")

In [ ]:
test = test_df["text"]
test_label = test_df["label"]

In [ ]:
sequences_test = tokenizer.texts_to_sequences(test)
test_data = pad_sequences(sequences_test,maxlen = 50)

In [ ]:
test_pred = model2.predict(test_data)

In [ ]:
new_pred = []
for i in test_pred:
    if i>0.5:
        new_pred.append("NAME")
    else:
        new_pred.append("NOTNAME")

In [ ]:
result = [(i,j)for i,j in zip(test,new_pred)]

In [ ]:
print(result)

In [ ]:
result

In [ ]:
new_pred = []
for i in prediction:
    if i>0.5:
        new_pred.append("NAME")
        print(i, new)
    else:
        new_pred.append("NOTNAME")

In [ ]:
result = [(i,j)for i,j in zip(new_pred,new)]
result